## Adding packages.

In [ ]:
using Pkg;

# packages to install, first time can take a while downloading, please be patient ...
packages=["HTTP","JSON","DataFrames","Plots","StatPlots","PlotThemes","GR","PyPlot","PyCall","LaTeXStrings","Plotly","PlotlyJS"]

for package in packages
    if get(Pkg.installed(),package,-1) == -1
        println(" getting package : ", package)
        Pkg.add(package)
    end
end

## Creating generic functions for Binance API data retrieval

In [ ]:
# show all data for a given value
function showAll(value)
    show(IOContext(stdout, :displaysize => (10^6, 10^6)), "text/plain", value)
end

using HTTP, JSON, Dates
import Printf.@sprintf

# base URL of the Binance API
const BINANCE_API_REST = "https://api.binance.com/api/v1/ticker/";

# function HTTP response 2 JSON
function r2j(response)
    JSON.parse(String(response))
end

# retrieve data from binance for "allPrices", "24hr", "allBookTickers"
function getBinanceDataFor(withSubject)
    r = HTTP.request("GET", string(BINANCE_API_REST, withSubject))
    r2j(r.body)
end

# current Binance market
function getBinanceMarket()
    r = HTTP.request("GET", "https://www.binance.com/exchange/public/product")
    r2j(r.body)["data"]
end

# binance get candlesticks/klines data
function getBinanceKline(symbol, startDateTime, endDateTime ; interval="1m")
    startTime = @sprintf("%.0d",Dates.datetime2unix(startDateTime) * 1000)
    endTime = @sprintf("%.0d",Dates.datetime2unix(endDateTime) * 1000)
    query = string("symbol=", symbol, "&interval=", interval, "&startTime=", startTime, "&endTime=", endTime) 
    r = HTTP.request("GET", string("https://api.binance.com/api/v1/klines?",  query))
    r2j(r.body)
end

# getFloatValueArray
getFloatValueArray(withKey, withDictArr) = map(dict -> parse(Float32, dict[withKey]) ,withDictArr)
getStringValueArray(withKey, withDictArr) = map(dict -> convert(String, dict[withKey]) ,withDictArr)
# filter 
filterOnRegex(matcher,withDictArr; withKey="symbol") = filter(x-> match(Regex(matcher), x[withKey]) != nothing, withDictArr)

In [ ]:
# Retrieve Symbols Data with interval 1 month "1M"

In [ ]:
using DataFrames;

# get 24H tickerdata of all assets
hr24 = getBinanceDataFor("24hr");
# previous post we determined that BTC currency was the most obvious currency to tade in, most avalable symbols, second h
hr24BTC = filterOnRegex("BTC\$", hr24)

symbols = getStringValueArray("symbol", hr24BTC);



# volume BTC ammount = volume * weightedAvgPrice
quoteVolume = getFloatValueArray("quoteVolume", hr24);
volume = getFloatValueArray("volume", hr24);
# Dataframe convenient structure to handle data in tabular format
df = DataFrame(basesymbol=basesymbols, symbol=symbols, lastPrice=lastPrice, priceChangePercent=priceChangePercent, quoteVolume=quoteVolume, volume=volume);
#sort!(df, :priceChangePercent; rev=false);

## Determine the TOP N highest traded symbols sorted descending

## Determine the assets from the TOP N, with the lowest value

In [ ]:
# get only symbols ending on BTC
hr24BTC = filterOnRegex("BTC\$", hr24)
hr24ETH = filterOnRegex("ETH\$", hr24)
hr24BNB = filterOnRegex("BNB\$", hr24)
hr24USDT = filterOnRegex("USDT\$", hr24)

println("BTC has : ",size(hr24BTC,1)," assets")
println("ETH has : ",size(hr24ETH,1)," assets")
println("BNB has : ",size(hr24BNB,1)," assets")
println("USDT has : ",size(hr24USDT,1)," assets")

In [ ]:
# get the latest currency values from base currencies
fdf = filter(row -> row[:symbol] in ["BTCUSDT","ETHUSDT","BNBUSDT"], df)
results = (fdf[:,[:symbol,:lastPrice]])
append!(results,DataFrame(symbol=["SDTUSDT"],lastPrice = [1.0]))
println("\n Get the \$ values from the base currencies\n")
unique!(results)

In [ ]:
using StatPlots
theme(:dark);

symbol_volume = by(df, :basesymbol) do d
    USDT_CONVERSION = filter(row -> row[:symbol] == string(d[:basesymbol][1], "USDT"), results)[:lastPrice]
    DataFrame(assetSize  = size(d[:symbol], 1), quoteVolume  = sum(d[:quoteVolume]), quoteVolumeUSDT  = sum(d[:quoteVolume])*USDT_CONVERSION/10^6)
    end;

show(sort!(symbol_volume, [:assetSize,:quoteVolumeUSDT]; rev=true))

Plots.plot(symbol_volume[:basesymbol], symbol_volume[:quoteVolumeUSDT],seriestype=:pie,title = "USDT trading distribution Binance")